In [1]:
# Imports 

from helper import ClassConstructor
from db.players_controller import PlayersController
from db.clubs_controller import ClubsController
from db.stadiums_controller import StadiumsController
from db.games_controller import GamesController

from logs_helper import LogsHandler

# Just int case
from pprint import pprint

In [2]:
# Class declare (statics)

class_const = ClassConstructor()
players_controller = PlayersController()
clubs_controller = ClubsController()
stadiums_controller = StadiumsController()
games_controller = GamesController()

logs_handler = LogsHandler()

In [3]:
# Select and transform stadium data into objects

stadiums_data = stadiums_controller.select_all_stadiums()
stadiums = class_const.stadiums(stadiums_data)

In [4]:
# Select players with contract

# get from db
p = players_controller.select_players_with_contract('2022') 

# transform into objects
players = class_const.players(p) 

In [5]:
# Serie A clubs selection & cast

serie_a = clubs_controller.select_serie_a_clubs()
serie_a_clubs = class_const.clubs(serie_a)
serie_a_clubs = class_const.add_players_to_clubs(serie_a_clubs, players)

In [6]:
# Set clubs formation

class_const.define_formation(serie_a_clubs)

In [7]:
# Define the clubs matches, this will return a list of lists with two differents teams

serie_a_schedule = class_const.define_schedule(serie_a_clubs)

In [8]:
# Return a list of games object with all the confrontations through the season

serie_a_games = class_const.prepare_games(serie_a_schedule, stadiums, 'Campeonato Brasileiro Serie A', 2022)

In [9]:
for game in serie_a_games:
    game.start()

In [10]:
# Here we started to insert games into database

In [11]:
# Prepare a stats data for game_stats 

for game in serie_a_games:
    stats_data = logs_handler.prepare_game_stats_logs_to_db(logs_handler.get_game_stats(game.logs, game.home, game.away))

    # Insert and get the id from the game stats
    s1 = games_controller.insert_game_stat_with_id_return(stats_data[0])
    s2 = games_controller.insert_game_stat_with_id_return(stats_data[1])

    # get the last games_stats id inserted
    game_ids = s1[0][0], s2[0][0]

    # prepare data to insert game
    prepare_game = logs_handler.prepare_game_logs_to_db(game.logs, game_ids)

    # insert games into the database
    games_controller.insert_games_list([prepare_game])